In [ ]:
import requests
import pandas as pd
import os
import sys
from datetime import datetime

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))

from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path
import src.esh_client as esh

base_url = get_base_url()
root_path = get_root_path()
example_name = sys.path[0].split(os.sep)[-2]


def execute_search_query(query):
    r = requests.post(
        f'{base_url}/v1/query/{example_name}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            for re in res['value']:
                pers_id = re['id']
                
            df = pd.DataFrame.from_dict(res['value'])[
                ['id','firstName', 'lastName', 'age']]
            df.rename(columns={'id':'ID','firstName': 'First Name',
                      'lastName': 'Last Name', 'age': 'Age'}, inplace=True)
            display(df)
            return pers_id
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())

## Create Person

In [ ]:
person = {
    "Person": [
        {
            "source": [
                {
                    "name": "systemA",
                    "type": "Person",
                    "sid": "GUIDP00099"
                }
            ],
            "firstName": "Maxi",
            "lastName": "Musti",
            "dob": "19820318",
            "age": "99",
            "complexion": "Fair",
            "ethnicity": "AFR",
            "height": 150,
            "gender": "Male",
            "countryOfBirth": "DE",
            "country": "AFR",
            "placeOfBirth": "Munich",
            "nationality": "DE"
        }
    ]
}

r = requests.post(
        f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=person)

if r.status_code == 200:
    print('Successful created!')
else:
    print('Error!')    

## Search for created Person (check)

In [ ]:
so = esh.EshObject(
    count=True,
    top=2,
    scope=['Person'],
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.Comparison(
                property=esh.Property(property=['lastName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='Mustim')
            )
        ]
    )
)
pers_id = execute_search_query(so.dict(exclude_none=True))

## Update Person

In [ ]:
person = {
    "Person": [
        {
            "source": [
                {
                    "name": "systemA",
                    "type": "Person",
                    "sid": "GUIDP00099"
                }
            ],
            "id": pers_id,
            "firstName": "Maxi",
            "lastName": "Mustim",
            "dob": "19820318",
            "age": "99",
            "complexion": "Fair",
            "ethnicity": "AFR",
            "height": 150,
            "gender": "Male",
            "countryOfBirth": "DE",
            "country": "AFR",
            "placeOfBirth": "Munich",
            "nationality": "DE"
        }
    ]
}

r = requests.put(
        f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=person)

if r.status_code == 200:
    print('Successful updated!')
else:
    print('Error!')
    print(r.text) 

## Delete Person

In [ ]:
person = {
    "Person": [
        {
            "id": pers_id
        }
    ]
}

r = requests.delete(
        f'{base_url}/v1/data/{example_name}{TENANT_SUFFIX}', json=person)

if r.status_code == 200:
    print('Successful deleted!')
else:
    print('Error!')
    print(r.text) 